Imports

In [1]:
import os
import json
import requests
from bs4 import BeautifulSoup

Utils


In [2]:
def extract(ancestor,selector, attribute=None, multiple=False):
    if multiple:
        if attribute:
            return [p[attribute]for p in ancestor.select(selector)]
        return [tag.get_text() for tag in ancestor.select(selector)]
    if attribute:
        try:
            return ancestor.select_one(selector)[attribute]
        except TypeError:
            return None
    try:
        return ancestor.select_one(selector).get_text()
    except AttributeError:
        return None
    

2. Sending HTTP request to acces first page with opinions 

In [ ]:
url = "https://www.ceneo.pl/84514582#tab=reviews"
response = requests.get(url)

In [ ]:
response.status_code


4. Purse the http cofr of first page with opinions 

In [ ]:
page_dom = BeautifulSoup(response.text, 'html.parser')

5. Extract required data from parse code

In [ ]:
opinions = page_dom.select("div.js_product-review:not(.user-post--highlight)")
for opinion in opinions:
    opinion_id = opinion["data-entry-id"]
    author = opinion.select_one("span.user-post__author-name").get_text()
    try:
        recommendation = opinion.select_one("span.user-post__author-recomendation > em").get_text()
    except AttributeError:
        recommendation = None
    stars = opinion.select_one("span.user-post__score-count").get_text()
    content = opinion.select_one("div.user-post__text").get_text()
    pros = [p.get_text() for p in opinion.select("div.review-feature__item--positive")]
    cons = [c.get_text() for c in opinion.select("div.review-feature__item--negative")]
    vote_yes = opinion.select_one("button.vote-yes")["data-total-vote"]
    vote_no = opinion.select_one("button.vote-no")["data-total-vote"]
    published = opinion.select_one("span.user-post__published > time:nth-child(1)")["datetime"]
    try:
        purchased = opinion.select_one("span.user-post__published > time:nth-child(2)")["datetime"]
    except TypeError:
        purchased = None
    print(opinion_id, author, recommendation, stars, content, pros, cons, vote_yes, vote_no, published, purchased, sep='\n')

6.If there are more pages, move to the next page and repeat steps 2-6 for it

In [ ]:
try:
    next_page = "https://www.ceneo.pl" + page_dom.select_one("a.pagination__next")["href"]
except TypeError:
    next_page = None

In [ ]:
product_id = "84514582"
next_page = f"https://www.ceneo.pl/{product_id}#tab=reviews"
all_opinions = []
while next_page:
    response = requests.get(next_page, headers=headers)
    if response.status_code == 200:
        print("OK")
        page_dom = BeautifulSoup(response.text, 'html.parser')
        opinions = page_dom.select("div.js_product-review:not(.user-post--highlight)")
        print(len(opinions))
        for opinion in opinions:
            single_opinion = {}
            single_opinion["opinion_id"] = opinion["data-entry-id"]
            single_opinion["author"] = extract(opinion, "span.user-post__author-name")
            single_opinion["recommendation"] = extract(opinion, "span.user-post__author-recomendation > em")
            single_opinion["stars"] = extract(opinion, "span.user-post__score-count")
            single_opinion["content"] = extract(opinion, "div.user-post__text")
            single_opinion["pros"] = [p.get_text() for p in opinion.select("div.review-feature__item--positive")]
            single_opinion["cons"] = [c.get_text() for c in opinion.select("div.review-feature__item--negative")]
            single_opinion["vote_yes"] = extract(opinion, "button.vote-yes", "data-total-vote")
            single_opinion["vote_no"] = extract(opinion, "button.vote-no""data-total-vote")
            single_opinion["published"] = extract("span.user-post__published > time:nth-child(1)", "datetime")
            try:
                next_page = "https://www.ceneo.pl" + page_dom.select_one("a.pagination__next")["href"]
            except TypeError: 
                next_page = None
    else:
        print("NOT OK")
print(all_opinions)

7.Save extracted data


In [ ]:
if not os.path.exists("./opinions"): 
    os.mkdir("./opinions")
with open(f"{product_id}.json", "w", encoding="UTF-8") as jf: 
    json.dump(all_opinions, jf, ensure_ascii=False, indent=4)